In [38]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, current_timestamp, expr, concat_ws, lit, floor, rand

In [39]:
spark = SparkSession.builder \
    .appName("Gerar Massa de Dados") \
    .master("spark://spark-master:7077") \
    .config("spark.jars", "/opt/bitnami/spark/jars/mysql-connector-j-8.0.33.jar") \
    .getOrCreate()

jdbc_url = "jdbc:mysql://mysql:3306/desafio"
jdbc_properties = {
    "user": "sparkuser",
    "password": "sparkpass",
    "driver": "com.mysql.cj.jdbc.Driver"
}

In [40]:
nomes = ["Wilian", "Natalia", "Gustavo", "Eder", "Wallace", "Wendy", "Caio", "Joao"]
sobrenomes = ["Gomes", "Santana", "Mateo", "Murada", "Vieira", "Cruz", "Monteiro", "Lanzana"]



In [41]:
associados = spark.range(1, 51).select(
    col("id"),
    expr(f"array({','.join([f"'{n}'" for n in nomes])})[cast(floor(rand() * {len(nomes)}) as int)]").alias("nome"),
    expr(f"array({','.join([f"'{s}'" for s in sobrenomes])})[cast(floor(rand() * {len(sobrenomes)}) as int)]").alias("sobrenome"),(floor(rand() * 50) + 18).alias("idade"),
    concat_ws("", expr("lower(nome)"), expr("lower(sobrenome)"), expr("cast(id as string)"), lit("@email.com")).alias("email")
)

print("Dados de 'associado':")
associados.show(5)


Dados de 'associado':
+---+-------+---------+-----+--------------------+
| id|   nome|sobrenome|idade|               email|
+---+-------+---------+-----+--------------------+
|  1|Natalia|   Murada|   35|nataliamurada1@em...|
|  2|  Wendy|  Lanzana|   64|wendylanzana2@ema...|
|  3| Wilian|  Santana|   55|wiliansantana3@em...|
|  4|   Joao|   Murada|   43|joaomurada4@email...|
|  5|   Joao|  Lanzana|   50|joaolanzana5@emai...|
+---+-------+---------+-----+--------------------+
only showing top 5 rows



In [45]:
contas = associados.select(
    col("id"),
    expr("if(rand() > 0.5, 'Corrente', 'Poupança')").alias("tipo_conta"),
    current_timestamp().alias("data_criacao"),
    col("id").alias("id_associado")
)
print("DataFrame 'conta' criado:")
contas.show(5)

DataFrame 'conta' criado:
+---+----------+--------------------+------------+
| id|tipo_conta|        data_criacao|id_associado|
+---+----------+--------------------+------------+
|  1|  Corrente|2025-03-29 01:42:...|           1|
|  2|  Corrente|2025-03-29 01:42:...|           2|
|  3|  Corrente|2025-03-29 01:42:...|           3|
|  4|  Poupança|2025-03-29 01:42:...|           4|
|  5|  Corrente|2025-03-29 01:42:...|           5|
+---+----------+--------------------+------------+
only showing top 5 rows



In [47]:
cartoes = associados.select(
    col("id"),
    (floor(rand() * 90000000) + 10000000).alias("num_cartao"),
    concat_ws(" ", col("nome"), col("sobrenome")).alias("nom_impresso"),
    col("id").alias("id_conta"),  # Vincula ao mesmo id da conta
    col("id").alias("id_associado")
)
print("DataFrame 'cartao' criado:")
cartoes.show(5)

DataFrame 'cartao' criado:
+---+----------+--------------+--------+------------+
| id|num_cartao|  nom_impresso|id_conta|id_associado|
+---+----------+--------------+--------+------------+
|  1|  68422410|Natalia Murada|       1|           1|
|  2|  39243616| Wendy Lanzana|       2|           2|
|  3|  32350136|Wilian Santana|       3|           3|
|  4|  20631846|   Joao Murada|       4|           4|
|  5|  94413878|  Joao Lanzana|       5|           5|
+---+----------+--------------+--------+------------+
only showing top 5 rows



In [48]:
movimentos = cartoes.select(
    col("id"),
    (rand() * 1000).cast("decimal(10,2)").alias("vlr_transacao"),
    expr("array('Compra', 'Saque', 'Depósito')[cast(floor(rand() * 3) as int)]").alias("des_transacao"),
    current_timestamp().alias("data_movimento"),
    col("id").alias("id_cartao")
)
print("DataFrame 'movimento' criado:")
movimentos.show(5)

DataFrame 'movimento' criado:
+---+-------------+-------------+--------------------+---------+
| id|vlr_transacao|des_transacao|      data_movimento|id_cartao|
+---+-------------+-------------+--------------------+---------+
|  1|       214.66|     Depósito|2025-03-29 01:45:...|        1|
|  2|        18.58|       Compra|2025-03-29 01:45:...|        2|
|  3|       481.46|        Saque|2025-03-29 01:45:...|        3|
|  4|       179.03|        Saque|2025-03-29 01:45:...|        4|
|  5|       947.10|        Saque|2025-03-29 01:45:...|        5|
+---+-------------+-------------+--------------------+---------+
only showing top 5 rows



In [ ]:
print("Gravando no MySQL...")

In [49]:
associados.write.jdbc(url=jdbc_url, table="associado", mode="append", properties=jdbc_properties)

In [51]:
contas.write.jdbc(url=jdbc_url, table="conta", mode="append", properties=jdbc_properties)

25/03/29 01:46:19 WARN TaskSetManager: Lost task 0.0 in stage 28.0 (TID 41) (172.19.0.4 executor 0): java.sql.BatchUpdateException: Duplicate entry '25' for key 'conta.PRIMARY'
	at com.mysql.cj.jdbc.exceptions.SQLError.createBatchUpdateException(SQLError.java:224)
	at com.mysql.cj.jdbc.ClientPreparedStatement.executeBatchSerially(ClientPreparedStatement.java:816)
	at com.mysql.cj.jdbc.ClientPreparedStatement.executeBatchInternal(ClientPreparedStatement.java:418)
	at com.mysql.cj.jdbc.StatementImpl.executeBatch(StatementImpl.java:795)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.savePartition(JdbcUtils.scala:753)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.$anonfun$saveTable$1(JdbcUtils.scala:904)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.$anonfun$saveTable$1$adapted(JdbcUtils.scala:903)
	at org.apache.spark.rdd.RDD.$anonfun$foreachPartition$2(RDD.scala:1039)
	at org.apache.spark.rdd.RDD.$anonfun$foreachPartition$2$adapted(RDD.sc

Py4JJavaError: An error occurred while calling o712.jdbc.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 28.0 failed 4 times, most recent failure: Lost task 0.3 in stage 28.0 (TID 47) (172.19.0.4 executor 0): java.sql.BatchUpdateException: Duplicate entry '25' for key 'conta.PRIMARY'
	at com.mysql.cj.jdbc.exceptions.SQLError.createBatchUpdateException(SQLError.java:224)
	at com.mysql.cj.jdbc.ClientPreparedStatement.executeBatchSerially(ClientPreparedStatement.java:816)
	at com.mysql.cj.jdbc.ClientPreparedStatement.executeBatchInternal(ClientPreparedStatement.java:418)
	at com.mysql.cj.jdbc.StatementImpl.executeBatch(StatementImpl.java:795)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.savePartition(JdbcUtils.scala:753)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.$anonfun$saveTable$1(JdbcUtils.scala:904)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.$anonfun$saveTable$1$adapted(JdbcUtils.scala:903

In [52]:
cartoes.write.jdbc(url=jdbc_url, table="cartao", mode="append", properties=jdbc_properties)

In [53]:
movimentos.write.jdbc(url=jdbc_url, table="movimento", mode="append", properties=jdbc_properties)

In [ ]:
print("Dados gerados e inseridos com sucesso!")

In [55]:
# Parar a sessão
spark.stop()